# Data retrieval from localbitcoins API

In [1]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone

In [2]:
def make_request(url_base,url):
    r = requests.get(url_base+url)
    file_json = r.json()
    return file_json

def json_to_pandas(file_json:dict):
    return pd.DataFrame.from_dict(file_json)


### Calculation exchange rate of reference to further math on pct_change:
def exchange_reference(df, date_reference):
    
    memory = {}
        
    for currency_code in df["currency_code"].unique():
        
        if currency_code != "VED":
            date_reference = date_reference
        else:
            date_reference = "2021-10-10"

    
        ## Select currency for calculations:
        df_exchange = df[df['currency_code'] == currency_code ]

        ## Select date to calculate reference

        df_date_reference = df_exchange.loc[(df_exchange.date).dt.date == pd.Timestamp(date_reference)]

        ### Averare exchange rate for day of reference:

        implicit_exchange_reference = df_date_reference.implicit_exchange.mean()
        
        memory[currency_code] = implicit_exchange_reference

    ### Percentual Variation

#     equation = ((df_exchange.implicit_exchange / implicit_exchange_reference)-1)*100
    
    
    return memory

## Currencies

In [3]:
url_base = 'https://localbitcoins.com/'
# Currencies:
url = "api/currencies"
currency_code = make_request(url_base,url)
df_currency_code = json_to_pandas(currency_code['data'])
# df_currency_code.info()
# df_currency_code.head()

In [4]:
## Extract country information related to currency
currency_name = []
currency_is_altcoin = []
for index, row in df_currency_code.iterrows():
    currency_name.append(row[0]["name"])
    currency_is_altcoin.append(row[0]["altcoin"])
    
## Reorganize on df

df_currency_code['currencies'] = currency_name
df_currency_code['altcoin'] = currency_is_altcoin
df_currency_code = df_currency_code.reset_index()

df_currency_code.head()

,index,currencies,currency_count,altcoin
0,ADA,Cardano (ADA),185,True
1,AED,United Arab Emirates Dirham (AED),185,False
2,AFN,Afghan Afghani (AFN),185,False
3,ALL,Albanian Lek (ALL),185,False
4,AMD,Armenian Dram (AMD),185,False


### Average price of btc by country (in local currency)

In [5]:
url = "bitcoinaverage/ticker-all-currencies/"
price_btc = make_request(url_base,url)
time_stamp = datetime.now().timestamp()

In [6]:
df_price_btc = json_to_pandas(price_btc)

#transpose data frame. Index is now by Country:

df_price_btc = df_price_btc.T
df_price_btc['time_stamp'] = time_stamp
df_price_btc = df_price_btc.reset_index()
print(df_price_btc.info())
df_price_btc.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       68 non-null     object 
 1   volume_btc  68 non-null     object 
 2   rates       68 non-null     object 
 3   avg_1h      28 non-null     object 
 4   avg_6h      59 non-null     object 
 5   avg_12h     65 non-null     object 
 6   avg_24h     68 non-null     object 
 7   time_stamp  68 non-null     float64
dtypes: float64(1), object(7)
memory usage: 4.4+ KB
None


,index,volume_btc,rates,avg_1h,avg_6h,avg_12h,avg_24h,time_stamp
0,VED,1.92505646,{'last': '163055.95'},165685.66,167218.58,166962.18,166576.30,1.642972e+09
1,COP,2.48638844,{'last': '137006000.00'},142576886.39,143545755.14,144146183.48,142882775.97,1.642972e+09
2,PEN,0.65049088,{'last': '134777.86'},136930.68,139634.29,139628.37,143181.82,1.642972e+09
3,ARS,0.19547648,{'last': '8273716.61'},8330434.85,7864967.54,7798748.30,7723142.19,1.642972e+09
4,USD,2.80816739,{'last': '37272.27'},36736.88,36994.71,37031.17,36419.90,1.642972e+09


## Merge and clean dataframe

In [7]:
df_market = df_price_btc.merge(df_currency_code, how='right')

# Drop the rows with NAN values in time_stamp
df_market.dropna(subset=['time_stamp'], inplace=True)

# Add date from time_stamp
df_market['date'] = datetime.fromtimestamp(time_stamp, tz=timezone.utc)
df_market = df_market[['time_stamp','date','index','currencies','volume_btc','avg_24h','avg_12h','avg_6h','avg_1h','altcoin']]
df_market.rename(columns = {'index':'currency_code'}, inplace = True)
print(df_market.info())
df_market.head(10)


### I should include some data formatting so info is stored in the right format

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 1 to 183
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   time_stamp     68 non-null     float64            
 1   date           68 non-null     datetime64[ns, UTC]
 2   currency_code  68 non-null     object             
 3   currencies     68 non-null     object             
 4   volume_btc     68 non-null     object             
 5   avg_24h        68 non-null     object             
 6   avg_12h        65 non-null     object             
 7   avg_6h         59 non-null     object             
 8   avg_1h         28 non-null     object             
 9   altcoin        68 non-null     bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(1), object(7)
memory usage: 5.4+ KB
None


,time_stamp,date,currency_code,currencies,volume_btc,avg_24h,avg_12h,avg_6h,avg_1h,altcoin
1,1.642972e+09,2022-01-23 21:09:30.189158+00:00,AED,United Arab Emirates Dirham (AED),0.01024068,131827.19,NaN,NaN,NaN,False
6,1.642972e+09,2022-01-23 21:09:30.189158+00:00,AOA,Angolan Kwanza (AOA),0.03906243,24320046.65,24413571.84,24160600.26,NaN,False
7,1.642972e+09,2022-01-23 21:09:30.189158+00:00,ARS,Argentine Peso (ARS),0.19547648,7723142.19,7798748.30,7864967.54,8330434.85,False
8,1.642972e+09,2022-01-23 21:09:30.189158+00:00,AUD,Australian Dollar (AUD),0.13940856,49136.73,50708.21,50308.03,50930.76,False
15,1.642972e+09,2022-01-23 21:09:30.189158+00:00,BDT,Bangladeshi Taka (BDT),0.02659449,2940872.34,2856445.74,2856445.74,NaN,False
21,1.642972e+09,2022-01-23 21:09:30.189158+00:00,BOB,Bolivian Boliviano (BOB),0.06016427,258992.75,263327.64,263050.24,NaN,False
22,1.642972e+09,2022-01-23 21:09:30.189158+00:00,BRL,Brazilian Real (BRL),0.39114279,198801.01,198719.75,198017.45,204520.37,False
26,1.642972e+09,2022-01-23 21:09:30.189158+00:00,BYN,Belarusian Ruble (BYN),0.22242855,96970.55,96606.24,96797.92,96833.96,False
28,1.642972e+09,2022-01-23 21:09:30.189158+00:00,CAD,Canadian Dollar (CAD),0.64901048,45613.76,46395.04,46284.55,47033.38,False
30,1.642972e+09,2022-01-23 21:09:30.189158+00:00,CHF,Swiss Franc (CHF),0.00907595,34282.92,34282.92,33253.06,NaN,False


In [8]:
## Adding avg_24h exchange rate USD/BTC for inmediate comparison:

df_market["avg_24h_usd"] = df_market[df_market['currency_code']=="USD"].avg_24h.tolist()[0]

In [9]:
### Formatting data types:

# Float values:
for column_label in ['volume_btc','avg_24h','avg_12h','avg_6h','avg_1h','avg_24h_usd']:

    df_market[column_label] = pd.to_numeric(df_market[column_label])

df_market.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 1 to 183
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   time_stamp     68 non-null     float64            
 1   date           68 non-null     datetime64[ns, UTC]
 2   currency_code  68 non-null     object             
 3   currencies     68 non-null     object             
 4   volume_btc     68 non-null     float64            
 5   avg_24h        68 non-null     float64            
 6   avg_12h        65 non-null     float64            
 7   avg_6h         59 non-null     float64            
 8   avg_1h         28 non-null     float64            
 9   altcoin        68 non-null     bool               
 10  avg_24h_usd    68 non-null     float64            
dtypes: bool(1), datetime64[ns, UTC](1), float64(7), object(2)
memory usage: 5.9+ KB


In [10]:
## Calculating implicit exchange rate:
df_market['implicit_exchange'] = df_market['avg_24h'].divide(df_market["avg_24h_usd"], fill_value = None)

In [11]:
# Drop the rows with NAN values in time_stamp
df_market.dropna(subset=['time_stamp'], inplace=True)

In [12]:
df_market.head(2)

,time_stamp,date,currency_code,currencies,volume_btc,avg_24h,avg_12h,avg_6h,avg_1h,altcoin,avg_24h_usd,implicit_exchange
1,1.642972e+09,2022-01-23 21:09:30.189158+00:00,AED,United Arab Emirates Dirham (AED),0.010241,131827.19,NaN,NaN,NaN,False,36419.9,3.619647
6,1.642972e+09,2022-01-23 21:09:30.189158+00:00,AOA,Angolan Kwanza (AOA),0.039062,24320046.65,24413571.84,24160600.26,NaN,False,36419.9,667.768079


## Save information in .csv and sql database

In [13]:
# Saving csv by timestamp:
df_market.to_csv(f'./data/csv/{time_stamp}.csv',index=False)
# Saving historical data by appending:
df_market.to_csv(f'./data/csv/historical_data.csv',index=False, mode = 'a')

In [14]:
# Append information on sqlite database

from sqlalchemy import Column, Integer, Float, String, DateTime,Boolean, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///data/CurrenciesDataBase.db')
print(type(engine))
df_market.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

<class 'sqlalchemy.engine.base.Engine'>


#### Calculation of pct_change taking 2021-10-03 as date of reference

In [15]:
# path_to_currencies = "sqlite:///data/CurrenciesDataBase.db"
# df_market = pd.read_sql('currencies_vs_btc', path_to_currencies)

# ### The pct calculations are not properly saved on the dataset unless I upload one previously saved. Why?
# # The pipeline defined solve the issue by saving the calculated data on a new dataset but the calculations
# # are done in the full dataset everytime the code runs.
# # This practice is not scalable

# # The pct could be calculated on the web interface for a given date selected by the users but this implies 
# # more processing time

In [16]:
# df = df_market.groupby(by=["date","currency_code"]).mean()
# df.reset_index(inplace=True)

# ### Dataset with implicit exchange of reference by currency:
# date_reference = "2021-10-03"
# exchange = exchange_reference(df, date_reference)
# exchange = pd.DataFrame.from_dict(exchange, orient="index")

# df['pct'] = df.apply(lambda x : ((x['implicit_exchange']/exchange.loc[x["currency_code"]])-1)*100, axis = 1 )

# ### Drop Null values:

# df.dropna(subset=['pct'], inplace=True)

In [17]:
# df.info()

In [18]:
# ### Saving on DB Version 2

# engine = create_engine('sqlite:///data/CurrenciesDataBase_V2.db')
# print(type(engine))
# df.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

In [19]:
now = dt.datetime.now()
print("Last Execution: ")
print(str(now))
print('Completed')

Last Execution: 
2022-01-23 22:09:30.554934
Completed
